# Data Analysis

In [1]:
%matplotlib inline

from mpmath import mp, mpf, sin, cos, sqrt, acos


from pathlib import Path
pathtohere = Path.cwd()
print(pathtohere)

C:\Users\02dba\Documents\GitHub\Diatomic-Molecule-Dissociation-by-Gravitational-Gradient\contents


In [2]:
from pyDataAnalysis.readData import *
from pyDataAnalysis.display import *

In [3]:
mp.dps = 25  # Decimal places of precision

In [7]:
def main():
    
    BH, particle1, moleculeLength = readPropertiesFile()
    particle1Coords = np.loadtxt(pathtohere / 'data/coords1.txt', delimiter=',', dtype=str)
    particle2Coords = np.loadtxt(pathtohere / 'data/coords2.txt', delimiter=',', dtype=str)

    
    # Convert the loaded data to high-precision floats
    high_precision_coords1 = np.array([mpf(x) for x in particle1Coords.flatten()], dtype=object)
    high_precision_coords2 = np.array([mpf(x) for x in particle2Coords.flatten()], dtype=object)

    # If the data is multi-dimensional, reshape it
    particle1Coords = high_precision_coords1.reshape(particle1Coords.shape)
    particle2Coords = high_precision_coords2.reshape(particle2Coords.shape)
    
    
    
    # Newtonian
    kg_to_m = 1. / 1.3466e+27
    sol_to_m = 1.98855e+30 * kg_to_m
    sol_to_kg = 1.988475e+30
    G = 6.6743e-11 # m^3 kg^-1 s^-2
    lightSpeed = 299_792_458. # m s^-1
    
    v1_esc = np.sqrt(2*G*BH.mass/sol_to_m * sol_to_kg / particle1Coords[0,2])
    v2_esc = np.sqrt(2*G*BH.mass/sol_to_m * sol_to_kg / particle2Coords[0,2])
    
    print('Newtonian escape velocities:')
    print(f'\tp1: {v1_esc/lightSpeed}c')
    print(f'\tp2: {v2_esc/lightSpeed}c')

    
    
    print(particle1Coords.shape)
    ## Temporary fix.
    renderFull = particle1Coords[0,2]>3e+11 or particle1Coords[0,2]<5e+10
    #displaySystem(BH, particle1Coords, particle2Coords, renderFull=renderFull)
        
    displayMolecule(particle1Coords, particle2Coords)
    
    displayCoordinateStats(BH,particle1Coords, particle2Coords)
    displayMoleculeAngle(particle1Coords, particle2Coords, moleculeLength)
    
    # print(particle1Coords[:10, 2], particle2Coords[:10,2])
    

In [8]:
if __name__=='__main__':
    main()

Newtonian escape velocities:
	p1: 0.9880256628149529320749757c
	p2: 0.9880256628149529320636722c
(10000, 5)


100%|████████████████████████████████████████████████████████████████████████| 10000/10000 [00:00<00:00, 422757.50it/s]
